# Exercício - Análise de Sentimentos

Gustavo Henrique da Silva Costa Araujo


20210155800

In [71]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [72]:
import pandas as pd



ETAPA 1 - Processamento dos dados.

In [73]:
dados = pd.read_csv('B2W-Reviews01.csv')
tokens_negacao = ['não', 'nem', 'nunca', 'jamais', 'tampouco']

C:\Users\PC\AppData\Local\Temp\ipykernel_5740\3244800104.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dados = pd.read_csv('B2W-Reviews01.csv')


In [74]:
dados.columns

Index(['submission_date', 'reviewer_id', 'product_id', 'product_name',
       'product_brand', 'site_category_lv1', 'site_category_lv2',
       'review_title', 'overall_rating', 'recommend_to_a_friend',
       'review_text', 'reviewer_birth_year', 'reviewer_gender',
       'reviewer_state'],
      dtype='object')

In [75]:
dados['review_text'] = dados['review_text'].astype(str)
textos = dados['review_text']
textos = textos.dropna()
textos = textos.str.lower()
textos = textos.to_numpy()

In [76]:
def adicionar_nao_prefixo(texto):
    palavras = texto.split()
    novo_texto = []
    negacao_ativa = False
    
    for palavra in palavras:
        if any(token in palavra.lower() for token in tokens_negacao):
            negacao_ativa = True
            nova_palavra = "NAO_" + palavra  
        elif any(p in palavra for p in ['.', ',', '?', '!', ';']):
            negacao_ativa = False
            nova_palavra = palavra  
        elif negacao_ativa:
            nova_palavra = "NAO_" + palavra
        else:
            nova_palavra = palavra  
        novo_texto.append(nova_palavra) 
        
    return ' '.join(novo_texto)

In [77]:
dados['review_text_modificado'] = dados['review_text'].apply(adicionar_nao_prefixo)


In [78]:
dados['review_text_modificado']

0         Estou contente com a compra entrega rápida o ú...
1         Por apenas R$1994.20,eu consegui comprar esse ...
2         SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3         MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4         A entrega foi no prazo, as americanas estão de...
                                ...                        
132368    Vale muito, estou usando no controle do Xbox e...
132369    Prático e barato, super indico o produto para ...
132370    Chegou antes do prazo previsto e corresponde a...
132371    Material fraco, poderia ser melhor. Ficou deve...
132372    Comprei esse produto, quando chegou estava com...
Name: review_text_modificado, Length: 132373, dtype: object

Etapa 2 - Divisão dos dados.

In [79]:
len(dados['review_text_modificado'])

132373

In [80]:
from sklearn.model_selection import train_test_split

X = dados['review_text_modificado']
y = dados['overall_rating']  # Usaremos 'overall_rating' como a classe de sentimentos

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)


In [81]:
len(X)

132373

In [82]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

# Crie um pipeline que inclui a vetorização do texto e o classificador Naive Bayes
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB())
])

# Treine o classificador com os dados de treinamento
pipeline.fit(X_treino, y_treino)


Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', MultinomialNB())])

In [83]:
# Faça previsões nos dados de teste
y_pred = pipeline.predict(X_teste)


In [84]:
print(X_teste)

328       Ele e muito bom ,bom, bom , bom , bom. BOM BOM...
83773     cumpre a funcao desejada melhor que a marca co...
117905    NAO_Não NAO_tem NAO_luz NAO_então NAO_não NAO_...
47675     Lindo painel,material de qualidade,forte e res...
51037     Muito boa a lente estabilização e foco maravil...
                                ...                        
5821      ótimo produto muito pratico o bebê adora ficar...
117296    Levando em consideração o preço o produto e ex...
64580     Produto de ótima qualidade e chegou uma semana...
5753      Produto dês ótima qualidade e com rápida efici...
39748     Até agora apresenta um desempenho impecável, c...
Name: review_text_modificado, Length: 26475, dtype: object


In [85]:
from sklearn.metrics import accuracy_score

# Calcule a acurácia comparando as previsões com os rótulos reais dos dados de teste
acuracia = accuracy_score(y_teste, y_pred)

print(f'Acurácia do classificador Naive Bayes com o prefixo _NÃO: {acuracia}')


Acurácia do classificador Naive Bayes com o prefixo _NÃO: 0.5673654390934845


In [86]:
X_teste_sem_prefixo = dados['review_text']

y_pred_sem_prefixo = pipeline.predict(X_teste_sem_prefixo)


In [87]:
len(y_pred_sem_prefixo)

132373

In [88]:
len(y_teste)

26475

nessa parte de cima não consegui finalizar a acuracia sem o prefixo pois a quantidade de índices estão divergentes e não consegui entender aonde que eles estão mudando

In [58]:
from sklearn.metrics import accuracy_score

# Calcule a acurácia comparando as previsões sem o prefixo "NAO_" com os rótulos reais dos dados de teste
acuracia_sem_prefixo = accuracy_score(y_teste, y_pred_sem_prefixo)

print(f'Acurácia do classificador Naive Bayes sem o prefixo "NAO_": {acuracia_sem_prefixo}')


ValueError: Found input variables with inconsistent numbers of samples: [26475, 132373]